In [1]:
from pathlib import Path
import requests
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import pandas as pd
import urllib
import re
from selenium.webdriver.common.by import By

In [4]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

chrome_options = Options()
chrome_options.add_argument("user-data-dir=selenium")
driver = webdriver.Chrome(options=chrome_options)

# Open the website
driver.get("https://classic.wyborcza.pl/archiwumGW/0,160510.html?searchForm=&datePeriod=0&initDate=2017-05-15&endDate=2022-12-31&publicationsString=1%3B5&author=&page=0&sort=OLDEST")

# Find all items by class
items = driver.find_elements(By.CLASS_NAME, "result-item-link")

# Print the text of each item
for item in items:
    try:
        item.click()
        time.sleep(1)
        driver.back()
    except:
        print(item.text)


# Close the browser
driver.quit()


ANDRZEJ RYSUJE
Macron u Merkel
WSZYSCY JESTEŚMY GIMBAMI
Co tak naprawdę robi pisarz, kiedy pisze
Nasz spacer po Kobierzynie


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x55db872c8f83 <unknown>
#1 0x55db86f81cf7 <unknown>
#2 0x55db86f93890 <unknown>
#3 0x55db86f88d54 <unknown>
#4 0x55db86f86bad <unknown>
#5 0x55db86f8a798 <unknown>
#6 0x55db86f8a8ec <unknown>
#7 0x55db86fcb2e2 <unknown>
#8 0x55db86ffb0b2 <unknown>
#9 0x55db86fc4b49 <unknown>
#10 0x55db86ffb4ce <unknown>
#11 0x55db8701a006 <unknown>
#12 0x55db86ffae53 <unknown>
#13 0x55db86fc2dd4 <unknown>
#14 0x55db86fc41de <unknown>
#15 0x55db8728d531 <unknown>
#16 0x55db87291455 <unknown>
#17 0x55db87279f55 <unknown>
#18 0x55db872920ef <unknown>
#19 0x55db8725d99f <unknown>
#20 0x55db872b6008 <unknown>
#21 0x55db872b61d7 <unknown>
#22 0x55db872c8124 <unknown>
#23 0x7fa3b3c97ada <unknown>
